In [0]:
table_name = "workspace.default.customer_segmentation_highacc"  # change if needed

df = spark.table(table_name)
df.printSchema()
display(df.limit(10))
print("Rows:", df.count())


root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- balance: long (nullable = true)
 |-- transaction_freq: long (nullable = true)
 |-- product_usage: long (nullable = true)
 |-- true_cluster: long (nullable = true)



customer_id customer_name age balance transaction_freq product_usage true_cluster CUST_1 Riya Verma 24 6624 90 3 0 CUST_2 Arjun Bose 21 6068 99 2 0 CUST_3 Saanvi Patel 30 2900 80 2 0 CUST_4 Aditya Nair 22 2449 64 3 0 CUST_5 Kavya Sharma 29 13868 93 2 0 CUST_6 Krishna Ghosh 23 3096 50 3 0 CUST_7 Rohit Das 24 11595 99 3 0 CUST_8 Kavya Pandey 23 13925 68 3 0 CUST_9 Krishna Menon 22 13928 53 2 0 CUST_10 Arjun Bose 23 11053 70 3 0

Rows: 500


In [0]:
from pyspark.sql.functions import col

# Keep only modeling columns (no IDs/names)
data = df.select("age","balance","transaction_freq","product_usage")

# Basic stats
display(data.summary())


summary age balance transaction_freq product_usage count 500 500 500 500 mean 40.128 369515.276 33.772 2.748 stddev 13.786651497700175 571350.4119744299 28.53805980993505 1.2520508226605824 min 20 1006 1 1 25% 30 10250 9 2 50% 38 83487 27 3 75% 48 393984 50 4 max 74 1964622 99 5

In [0]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(
    inputCols=["age","balance","transaction_freq","product_usage"],
    outputCol="features_raw"
)
assembled = assembler.transform(data)

scaler = StandardScaler(inputCol="features_raw", outputCol="features", withMean=True, withStd=True)
scaled = scaler.fit(assembled).transform(assembled)

display(scaled.select("features").limit(5))


features {"type":"1","size":null,"indices":null,"values":["-1.1698272058802948","-0.6351466077462821","1.9702811044086852","0.20126978509107335"]} {"type":"1","size":null,"indices":null,"values":["-1.3874289926883856","-0.6361197408504985","2.2856494251684167","-0.5974198382862081"]} {"type":"1","size":null,"indices":null,"values":["-0.7346236322641131","-0.6416644992572568","1.619871859120095","-0.5974198382862081"]} {"type":"1","size":null,"indices":null,"values":["-1.3148950637523553","-0.6424538572248856","1.0592170666583502","0.20126978509107335"]} {"type":"1","size":null,"indices":null,"values":["-0.8071575612001434","-0.6224678735611316","2.0754038779952624","-0.5974198382862081"]}

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette", distanceMeasure="squaredEuclidean")

results = []
for k in [2,3,4,5,6]:
    km = KMeans(featuresCol="features", k=k, seed=42)
    model = km.fit(scaled)
    pred = model.transform(scaled)
    sil = evaluator.evaluate(pred)
    results.append((k, sil))

# Show silhouette by K
results


[(2, 0.5850832826758854),
 (3, 0.5822811858330099),
 (4, 0.5737719913376125),
 (5, 0.5363914678381941),
 (6, 0.5683357656975562)]

In [0]:
best_k = 3  # change if your silhouette table suggests another K

km = KMeans(featuresCol="features", k=best_k, seed=42)
km_model = km.fit(scaled)
pred_all = km_model.transform(scaled)

# Attach cluster back to original rows
# Re-attach identifiers for reporting
pred_with_ids = km_model.transform(
    scaler.fit(assembler.transform(df.select("age","balance","transaction_freq","product_usage"))).transform(
        assembler.transform(df.select("age","balance","transaction_freq","product_usage"))
    )
)

# Better: run scaler/assembler once on df and join with ids
from pyspark.sql.functions import monotonically_increasing_id
df_id = df.withColumn("id", monotonically_increasing_id())
assembled_id = assembler.transform(df_id)
scaled_id = scaler.fit(assembled_id).transform(assembled_id)
pred = km_model.transform(scaled_id)

# 'pred' now has all original columns + cluster
display(pred.select("customer_id","customer_name","age","balance","transaction_freq","product_usage","prediction").limit(20))


customer_id customer_name age balance transaction_freq product_usage prediction CUST_1 Riya Verma 24 6624 90 3 2 CUST_2 Arjun Bose 21 6068 99 2 2 CUST_3 Saanvi Patel 30 2900 80 2 2 CUST_4 Aditya Nair 22 2449 64 3 2 CUST_5 Kavya Sharma 29 13868 93 2 2 CUST_6 Krishna Ghosh 23 3096 50 3 2 CUST_7 Rohit Das 24 11595 99 3 2 CUST_8 Kavya Pandey 23 13925 68 3 2 CUST_9 Krishna Menon 22 13928 53 2 2 CUST_10 Arjun Bose 23 11053 70 3 2 CUST_11 Ishita Verma 33 8531 89 2 2 CUST_12 Ananya Joshi 34 2619 53 2 0 CUST_13 Vivaan Pillai 23 7868 59 5 2 CUST_14 Sneha Patel 31 5713 59 3 2 CUST_15 Aarav Kumar 31 5998 64 4 2 CUST_16 Krishna Verma 31 9461 39 5 2 CUST_17 Riya Kumar 25 3014 99 5 2 CUST_18 Aditya Verma 28 6347 85 4 2 CUST_19 Kavya Das 23 9017 86 2 2 CUST_20 Kavya Nair 23 1689 62 4 2

In [0]:
import pyspark.sql.functions as F

profile = (pred
    .groupBy("prediction")
    .agg(
        F.count("*").alias("customers"),
        F.avg("age").alias("avg_age"),
        F.avg("balance").alias("avg_balance"),
        F.avg("transaction_freq").alias("avg_txn_freq"),
        F.avg("product_usage").alias("avg_products")
    )
    .orderBy("prediction")
)
display(profile)


prediction customers avg_age avg_balance avg_txn_freq avg_products 0 202 39.94059405940594 117008.21782178218 23.57920792079208 2.3217821782178216 1 126 58.56349206349206 1262744.349206349 4.992063492063492 2.0873015873015874 2 172 26.843023255813954 11722.03488372093 66.82558139534883 3.7325581395348837

In [0]:
# Bring profile to pandas to label segments
pdf = profile.toPandas()

# Heuristic naming based on averages (you can tweak thresholds)
names = {}
for _, r in pdf.iterrows():
    c = int(r["prediction"])
    if r["avg_balance"] > pdf["avg_balance"].mean()*1.5 and r["avg_txn_freq"] < pdf["avg_txn_freq"].mean():
        names[c] = "Wealth Guardians"          # high balance, low activity
    elif r["avg_txn_freq"] > pdf["avg_txn_freq"].mean() and r["avg_balance"] < pdf["avg_balance"].mean():
        names[c] = "Everyday Transactors"      # low balance, high activity
    else:
        names[c] = "Growth Seekers"            # mid balance, medium-high activity

names


{0: 'Growth Seekers', 1: 'Wealth Guardians', 2: 'Everyday Transactors'}

In [0]:
from pyspark.sql.functions import when, lit, col

# pred must exist from the previous cell (has 'prediction' + original cols)
pred_named = pred.withColumn("segment", lit(None).cast("string"))

for cid, label in names.items():   # names is the {cluster_id: "Label"} dict from Cell 7
    pred_named = pred_named.withColumn(
        "segment",
        when(col("prediction") == int(cid), lit(label)).otherwise(col("segment"))
    )

display(pred_named.select("customer_id","customer_name","age","balance",
                          "transaction_freq","product_usage","prediction","segment").limit(20))


customer_id customer_name age balance transaction_freq product_usage prediction segment CUST_1 Riya Verma 24 6624 90 3 2 Everyday Transactors CUST_2 Arjun Bose 21 6068 99 2 2 Everyday Transactors CUST_3 Saanvi Patel 30 2900 80 2 2 Everyday Transactors CUST_4 Aditya Nair 22 2449 64 3 2 Everyday Transactors CUST_5 Kavya Sharma 29 13868 93 2 2 Everyday Transactors CUST_6 Krishna Ghosh 23 3096 50 3 2 Everyday Transactors CUST_7 Rohit Das 24 11595 99 3 2 Everyday Transactors CUST_8 Kavya Pandey 23 13925 68 3 2 Everyday Transactors CUST_9 Krishna Menon 22 13928 53 2 2 Everyday Transactors CUST_10 Arjun Bose 23 11053 70 3 2 Everyday Transactors CUST_11 Ishita Verma 33 8531 89 2 2 Everyday Transactors CUST_12 Ananya Joshi 34 2619 53 2 0 Growth Seekers CUST_13 Vivaan Pillai 23 7868 59 5 2 Everyday Transactors CUST_14 Sneha Patel 31 5713 59 3 2 Everyday Transactors CUST_15 Aarav Kumar 31 5998 64 4 2 Everyday Transactors CUST_16 Krishna Verma 31 9461 39 5 2 Everyday Transactors CUST_17 Riya Kumar 25 3014 99 5 2 Everyday Transactors CUST_18 Aditya Verma 28 6347 85 4 2 Everyday Transactors CUST_19 Kavya Das 23 9017 86 2 2 Everyday Transactors CUST_20 Kavya Nair 23 1689 62 4 2 Everyday Transactors

In [0]:
from pyspark.sql.functions import create_map, lit, col

offer_map = {
    "Wealth Guardians":   "Premium savings, wealth advisory, HNI credit card, family office services",
    "Everyday Transactors":"UPI cashback, debit card rewards, bill-pay discounts, micro-loans",
    "Growth Seekers":     "SIP/Mutual funds starter pack, credit card with EMI offers, pre-approved personal loan"
}

# Build a Spark map: segment -> offer
kv = []
for seg, offer in offer_map.items():
    kv += [lit(seg), lit(offer)]
seg2offer = create_map(*kv)

offer_df = pred_named.withColumn("recommended_offer", seg2offer[col("segment")])

display(offer_df.select("customer_id","customer_name","segment","recommended_offer").limit(20))


customer_id customer_name segment recommended_offer CUST_1 Riya Verma Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_2 Arjun Bose Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_3 Saanvi Patel Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_4 Aditya Nair Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_5 Kavya Sharma Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_6 Krishna Ghosh Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_7 Rohit Das Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_8 Kavya Pandey Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_9 Krishna Menon Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_10 Arjun Bose Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_11 Ishita Verma Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_12 Ananya Joshi Growth Seekers SIP/Mutual funds starter pack, credit card with EMI offers, pre-approved personal loan CUST_13 Vivaan Pillai Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_14 Sneha Patel Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_15 Aarav Kumar Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_16 Krishna Verma Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_17 Riya Kumar Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_18 Aditya Verma Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_19 Kavya Das Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans CUST_20 Kavya Nair Everyday Transactors UPI cashback, debit card rewards, bill-pay discounts, micro-loans

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette", distanceMeasure="squaredEuclidean")
silhouette = evaluator.evaluate(pred)
print(f"Silhouette score (k={best_k}): {silhouette:.3f}")


Silhouette score (k=3): 0.582


In [0]:
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol="features", outputCol="pca2")
pca_model = pca.fit(scaled_id)
pca_df = pca_model.transform(scaled_id)
pca_pred = km_model.transform(pca_df)

display(pca_pred.select("pca2","prediction").limit(1000))
# In the chart settings, choose pca2[0] on X, pca2[1] on Y, color by prediction.


pca2 prediction {"type":"1","size":null,"indices":null,"values":["2.110189457482432","-0.4020976075912568"]} 2 {"type":"1","size":null,"indices":null,"values":["2.101633070894629","-1.1700405369486844"]} 2 {"type":"1","size":null,"indices":null,"values":["1.379825617031275","-1.0249730948693152"]} 2 {"type":"1","size":null,"indices":null,"values":["1.7079662251209344","-0.36778928988819615"]} 2 {"type":"1","size":null,"indices":null,"values":["1.6549284338103418","-1.0585748230332999"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4038661978863085","-0.32335378121097647"]} 2 {"type":"1","size":null,"indices":null,"values":["2.274634662899051","-0.41933580067994486"]} 2 {"type":"1","size":null,"indices":null,"values":["1.7319747170071085","-0.3571800131267826"]} 2 {"type":"1","size":null,"indices":null,"values":["1.1909004677709294","-1.046071485051606"]} 2 {"type":"1","size":null,"indices":null,"values":["1.7720235330909393","-0.3639255443256958"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4209599713151733","-1.0072923080677376"]} 2 {"type":"1","size":null,"indices":null,"values":["0.709953125806892","-0.9166199192612756"]} 0 {"type":"1","size":null,"indices":null,"values":["2.1696922426545986","1.0836988369012792"]} 2 {"type":"1","size":null,"indices":null,"values":["1.2431089635724197","-0.2504901789576768"]} 2 {"type":"1","size":null,"indices":null,"values":["1.6372533063679409","0.4503360343970864"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4658390754479198","1.2230978027195756"]} 2 {"type":"1","size":null,"indices":null,"values":["2.8424945599645293","1.010289566811337"]} 2 {"type":"1","size":null,"indices":null,"values":["2.153604736281543","0.367432333808683"]} 2 {"type":"1","size":null,"indices":null,"values":["1.7733527128489355","-1.1147595062950941"]} 2 {"type":"1","size":null,"indices":null,"values":["1.9308120455479794","0.35999079518741633"]} 2 {"type":"1","size":null,"indices":null,"values":["1.1604432770554047","-0.26822679398800836"]} 2 {"type":"1","size":null,"indices":null,"values":["1.7296968277819467","1.1602397303380625"]} 2 {"type":"1","size":null,"indices":null,"values":["1.9767057500150385","1.1234428432992447"]} 2 {"type":"1","size":null,"indices":null,"values":["0.7985415304142417","-0.9652905073088955"]} 0 {"type":"1","size":null,"indices":null,"values":["1.362722913280558","-0.31168113709674206"]} 2 {"type":"1","size":null,"indices":null,"values":["2.2629116018357722","1.0765917344793907"]} 2 {"type":"1","size":null,"indices":null,"values":["1.0573046763315543","-1.0492412995154539"]} 0 {"type":"1","size":null,"indices":null,"values":["1.2550344932310527","-0.24311156358946104"]} 2 {"type":"1","size":null,"indices":null,"values":["1.2823624694022768","-1.0477002727490405"]} 2 {"type":"1","size":null,"indices":null,"values":["1.3728808635487282","-0.2947986608776251"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4446663478439694","-0.27604909345692424"]} 2 {"type":"1","size":null,"indices":null,"values":["2.4435368545348837","-0.46962604746221565"]} 2 {"type":"1","size":null,"indices":null,"values":["2.139708214413079","-0.39607019586825853"]} 2 {"type":"1","size":null,"indices":null,"values":["0.9980071824037784","-0.26069329857961293"]} 0 {"type":"1","size":null,"indices":null,"values":["1.7058126814301562","1.170838970655033"]} 2 {"type":"1","size":null,"indices":null,"values":["1.7728852852519985","-0.36865915022291723"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4273255861386545","-0.27089077206175094"]} 2 {"type":"1","size":null,"indices":null,"values":["1.168581662845464","0.5080978021115675"]} 2 {"type":"1","size":null,"indices":null,"values":["2.57143464675372","1.0244613761431198"]} 2 {"type":"1","size":null,"indices":null,"values":["1.2621775832650362","-0.28643124447053564"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4361782814890192","1.2465345020628935"]} 2 {"type":"1","size":null,"indices":null,"values":["1.4098260376